In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
from PIL import Image, ImageDraw
import os
import cv2
import random
import json
import time

from landmark_detector import KeypointDetector

In [ ]:
DATA_DIR = '/home/gpu2/hdd/dan/CelebA/val/'
MODEL_PATH = 'model.pb'

In [ ]:
keypoint_detector = KeypointDetector(MODEL_PATH, gpu_memory_fraction=0.25, visible_device_list='1')

# Get images and bounding boxes

In [ ]:
metadata = []
NUM_ANNOTATIONS = 1000
for n in os.listdir(os.path.join(DATA_DIR, 'annotations'))[:NUM_ANNOTATIONS]:
    a = json.load(open(os.path.join(DATA_DIR, 'annotations', n)))
    image_name = a['filename']
    path = os.path.join(DATA_DIR, 'images', image_name)
    box = a['box']
    metadata.append((path, box))

# Show detections

In [ ]:
def draw_on_image(image, keypoints):

    image_copy = image.copy()
    draw = ImageDraw.Draw(image_copy, 'RGBA')

    for y, x in keypoints:
        draw.ellipse([
            (x - 2.0, y - 2.0),
            (x + 2.0, y + 2.0)
        ], outline='red')

    return image_copy

In [ ]:
i = random.randint(0, len(metadata) - 1)

# load and preprocess an image
image_array = cv2.imread(metadata[i][0])
image_array = cv2.cvtColor(image_array, cv2.COLOR_BGR2RGB)
image = Image.fromarray(image_array)

box = metadata[i][1]
crop = image.crop((box['xmin'], box['ymin'], box['xmax'], box['ymax']))
crop = crop.resize((112, 128))
crop

In [ ]:
crop_array = np.array(crop)
keypoints = keypoint_detector(crop_array)
draw_on_image(crop, keypoints)

# Measure speed

In [ ]:
times = []
for _ in range(110):
    start = time.perf_counter()
    _ = keypoint_detector(crop_array)
    times.append(time.perf_counter() - start)
    
times = np.array(times)
times = times[10:]
print(times.mean(), times.std())